# 并行前缀算法的实现
简单来说，并行前缀算法要做的事情就是给定一种二元运算符，以及一组元素，计算它们的运算结果。我们希望保留部分运算结果，并行前缀算法的目的就是生成n次求和运算的结果所组成的集合。

下面实现的是一个朴素并行前缀算法，它不仅假设输入元素个数为n，还进一步假设n是2的k次幂，同时在n个处理器（或n个线程）上并行运行这个算法。其时间复杂度为O(log n)。


In [2]:
import pycuda.autoinit
import pycuda.driver as drv
import numpy as np
from pycuda import gpuarray
from pycuda.compiler import SourceModule
from time import time
# this is a naive parallel prefix-sum kernel that uses shared memory
naive_ker = SourceModule("""
__global__ void naive_prefix(double *vec, double *out)
{
     __shared__ double sum_buf[1024];     
     int tid = threadIdx.x;     
     sum_buf[tid] = vec[tid];
     
     // begin parallel prefix sum algorithm
     int iter = 1;
     for (int i=0; i < 10; i++)
     {
         __syncthreads();
         if (tid >= iter )
         {
             sum_buf[tid] = sum_buf[tid] + sum_buf[tid - iter];            
         }
         
         iter *= 2;
     }
         
    __syncthreads();
    out[tid] = sum_buf[tid];
    __syncthreads();
        
}
""")
naive_gpu = naive_ker.get_function("naive_prefix")
    

if __name__ == '__main__':    
    testvec = np.random.randn(1024).astype(np.float64)
    testvec_gpu = gpuarray.to_gpu(testvec)
    
    outvec_gpu = gpuarray.empty_like(testvec_gpu)

    naive_gpu( testvec_gpu, outvec_gpu, block=(1024,1,1), grid=(1,1,1))
    
    total_sum = sum(testvec)
    total_sum_gpu = outvec_gpu[-1].get()

    print ("Does our kernel work correctly? : {}".format(np.allclose(total_sum_gpu , total_sum) ))

-2.8472944374569846 -2.847294437456995
Does our kernel work correctly? : True
